In [1]:
from azureml.core import Workspace, Experiment
current_sid = "9a7511b8-150f-4a58-8528-3e7d50216c31"
current_rg = "aml-quickstarts-142000"
current_ws = "quick-starts-ws-142000"
ws = Workspace.get(name=current_ws, resource_group=current_rg, subscription_id=current_sid  )
exp = Experiment(workspace=ws, name="project-hyperdrive-new")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-142000
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-142000


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "auto-ml-gyula-03"
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("This compute exists")
except ComputeTargetException:
    print("Building my own")
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes = 4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

This compute exists
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [13]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive import choice
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.core import ScriptRunConfig
import os

# Specify parameter sampler
ps = {
    '--C' : choice( 0.5, 1.0, 1.5, 1.8),
    '--max_iter' : choice( 50, 100, 150, 200)
}
samp = RandomParameterSampling(parameter_space = ps)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = ".",
                        entry_script="train.py", 
                        compute_target = compute_target,
                        script_params = {'--C': 1.0, '--max_iter':100})

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est, 
                                        hyperparameter_sampling = samp, 
                                        primary_metric_name = "Accuracy",
                                        primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                        max_total_runs = 20, 
                                        max_concurrent_runs = 4)

In [14]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.train.hyperdrive import HyperDriveRun
from azureml.core.experiment import Experiment

hyperdrive_run = exp.submit(hyperdrive_config)
widget1 = RunDetails(hyperdrive_run)
widget1.show()

The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--C', '--max_iter'] is the list of overridden parameter(s).


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [15]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
#parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best run id: ', best_run.id, ', best run metrics: ', best_run_metrics)

Best run id:  HD_bcc7b0ea-8c12-4a22-a517-6e3cbfe2ef20_0 , best run metrics:  {'Regularization Strength:': 1.0, 'Max iterations:': 150, 'Accuracy': 0.9072837632776934}


In [17]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
model = best_run.register_model(model_name='sklearn-logreg-hdr',
                               model_path="./outputs/model.pkl",
                               model_framework = Model.Framework.SCIKITLEARN,
                               model_framework_version='0.19.1')


In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds2 = TabularDatasetFactory.from_delimited_files(path, validate=True, include_path=False, infer_column_types=True,
                                                set_column_types=None, separator=',', header=True, partition_format=None, support_multi_line=False, empty_as_string=False)


In [4]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
# Use the clean_data function to clean your data.
x, y = clean_data(ds2)
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42)

clean_training_data = pd.DataFrame(data=pd.concat((x_train, y_train), axis=1), columns = x_train.columns.to_list()+['y'])
clean_validation_data = pd.DataFrame(data=pd.concat((x_test, y_test), axis=1), columns = x_test.columns.to_list()+['y'])
clean_full_data = pd.DataFrame(data=pd.concat((x, y), axis=1), columns = x.columns.to_list()+['y'])
clean_full_data.size


1318000

In [5]:
clean_training_data.describe()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
count,26360.000000,26360.000000,26360.000000,26360.000000,26360.000000,26360.000000,26360.000000,26360.000000,26360.000000,26360.000000,...,26360.000000,26360.000000,26360.000000,26360.000000,26360.000000,26360.000000,26360.000000,26360.000000,26360.000000,26360.000000
mean,40.004666,0.604666,0.000114,0.523065,0.150379,6.608953,2.982436,258.247193,2.560357,962.083536,...,0.365971,0.099886,0.054514,0.148141,0.230956,0.000417,0.128073,0.295334,0.042678,0.111950
std,10.426321,0.488932,0.010668,0.499477,0.357450,2.044491,1.409227,258.577678,2.777905,187.873072,...,0.481711,0.299854,0.227034,0.355247,0.421452,0.020424,0.334177,0.456201,0.202135,0.315311
min,17.000000,0.000000,0.000000,0.000000,0.000000,3.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2.000000,102.000000,1.000000,999.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,38.000000,1.000000,0.000000,1.000000,0.000000,6.000000,3.000000,179.000000,2.000000,999.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,47.000000,1.000000,0.000000,1.000000,0.000000,8.000000,4.000000,319.000000,3.000000,999.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,98.000000,1.000000,1.000000,1.000000,1.000000,12.000000,5.000000,4918.000000,56.000000,999.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
dstore = ws.get_default_datastore()
clean_training_data.to_csv("data/clean_train_data.csv", index=False)
clean_validation_data.to_csv("data/clean_validation_data.csv", index=False)
clean_full_data.to_csv("data/clean_full_data.csv", index=False)
dstore.upload(src_dir="./data", target_path="bankmarketing", overwrite=True, show_progress=True)

Uploading an estimated of 3 files
Uploading ./data/clean_full_data.csv
Uploaded ./data/clean_full_data.csv, 1 files out of an estimated total of 3
Uploading ./data/clean_train_data.csv
Uploaded ./data/clean_train_data.csv, 2 files out of an estimated total of 3
Uploading ./data/clean_validation_data.csv
Uploaded ./data/clean_validation_data.csv, 3 files out of an estimated total of 3
Uploaded 3 files


$AZUREML_DATAREFERENCE_b209a29d08da48e8ae71d76df08fa119

In [7]:
train_data = TabularDatasetFactory.from_delimited_files(path=dstore.path('bankmarketing/clean_train_data.csv'))

In [8]:
validation_data = TabularDatasetFactory.from_delimited_files(path=dstore.path('bankmarketing/clean_validation_data.csv'))

In [9]:
full_data = TabularDatasetFactory.from_delimited_files(path=dstore.path('bankmarketing/clean_full_data.csv'))

In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config1 = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    compute_target = compute_target,
    primary_metric='accuracy',
    training_data= train_data,
    validation_data= validation_data,
    label_column_name= 'y')
#for 1M+ rows we do not use cross validation, however it can be tried
automl_config2 = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    compute_target = compute_target,
    primary_metric='accuracy',
    training_data= full_data,
    label_column_name= 'y',
    n_cross_validations=3
)


In [ ]:
# Submit your automl run
from azureml.core.experiment import Experiment
auto_exp1 = Experiment(ws, "automl-train-validation")
run = auto_exp1.submit(automl_config1, show_output=True)


Running on remote.
No run_configuration provided, running on auto-ml-gyula-03 with default configuration
Running on remote compute: auto-ml-gyula-03
Parent Run ID: AutoML_15c7edad-74bd-4c49-be89-44e901dd857e



In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###